In [4]:
print('Starting generic python imports')
from pathlib import Path
import numpy as np
import os
from time import time, sleep
import sys
from munch import Munch
import random

print('Doing FKMC imports')
from FKMC.montecarlo import *

## overall input parameters

#put the repeat axis at the top so that I can watch the data come in in realtime!
repeats = 5
Us = [0.5,1,1.5,2,5,10,30]
Ts = Ts = [100, 50, 10, 5, 3, 2, 1, 0.1]
J = 5
Ns = [30,60,90,120,150]
alpha = 1.25

MCMC_params = Munch(
        state = None, #this gets filled in later because it depends on N
        N_steps = int(1e5),
        N_burn_in = int(1e4), 
        thin = 100,
        logger = Eigenspectrum_IPR_all(bins = 1000, limit = 10),
        proposal = p_multi_site_uniform_reflect,
        accept_function = perturbation_accept,
        warnings = True,
    )


total_jobs = repeats * len(Us) * len(Ts)

#give information to the dispatch script
batch_params = Munch(total_jobs = total_jobs, Us = Us, Ts = Ts, Ns = Ns, repeats = repeats, structure_names = ('repeats','Us','Ts'))

print('Getting environment variables')
slurm_job_id = int(os.getenv('SLURM_ARRAY_JOB_ID', -1))
slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_ID', random.random()*total_jobs))
max_slurm_task_id = int(os.getenv('SLURM_ARRAY_TASK_MAX', total_jobs - 1))

##sleep if necessary
debug = (slurm_job_id == -1)
sleeptime = np.random.random() * (30)
print(f'Waiting for {sleeptime:.0f} seconds to randomise the finish time')
if not debug: sleep(sleeptime)

##using the value of slurm_task_id, decide what parameters this job will have
repeat_i, j = divmod(slurm_task_id, len(Us) * len(Ts))
U_i, T_i = divmod(j, len(Ts))

U = Us[U_i]
T = Ts[T_i]
make_initial_state = lambda N: np.random.choice(np.array([0,1], dtype = np.float64), size = N)

logs = np.empty(shape = len(Ns), dtype = object)
    
for i, N in enumerate(Ns):
    parameters = Munch(t = 1, alpha = alpha, mu = 0, beta = 1/T, J = J, U = U, normalise = True)
    MCMC_params = Munch(
            state = make_initial_state(N),
            N_steps = int(1e4) if not debug else 100,
            N_burn_in = int(1e3) if not debug else 0,
            thin = 100 if not debug else 1,
            logger = Eigenspectrum_IPR_all(bins = 2000, limit = 20),
            proposal = p_multi_site_uniform_reflect,
            accept_function = perturbation_accept,
            warnings = True,
        )
    print(f'starting N = {N}')
    parameters.N_sites = N
    MCMC_params.state = np.random.choice(np.array([0,1], dtype = np.float64), size = N)
    
    t0 = time()
    logs[i] = FK_mcmc(**MCMC_params, parameters = parameters)
    logs[i].time = time() - t0
    


    filepath = Path(f'./{slurm_task_id}.npz') if not debug else Path(f'./test_save.npz')

    print(f'Saving in {filepath}')
    np.savez_compressed(filepath, 
        **batch_params, parameters = parameters, MCMC_params = MCMC_params, logs = logs, allow_pickle = True,
        desc = ''
    )
print('done')

Starting generic python imports
Doing FKMC imports
Getting environment variables
Waiting for 20 seconds to randomise the finish time
starting N = 30
0% through after 0.00m
10% through after 0.00m
20% through after 0.00m
30% through after 0.00m
40% through after 0.00m
50% through after 0.00m
60% through after 0.01m
70% through after 0.01m
80% through after 0.01m
90% through after 0.01m
Warning, p_acc = 0.713, Munch({'t': 1, 'alpha': 1.25, 'mu': 0, 'beta': 0.1, 'J': 5, 'U': 2, 'normalise': True, 'N_sites': 30, 'J_matrix': 'suppressed for brevity'})
Saving in test_save.npz
starting N = 60
0% through after 0.00m
10% through after 0.00m
20% through after 0.01m
30% through after 0.01m
40% through after 0.01m
50% through after 0.02m
60% through after 0.02m
70% through after 0.02m
80% through after 0.03m
90% through after 0.03m
Warning, p_acc = 0.6513333333333333, Munch({'t': 1, 'alpha': 1.25, 'mu': 0, 'beta': 0.1, 'J': 5, 'U': 2, 'normalise': True, 'N_sites': 60, 'J_matrix': 'suppressed for b